#### 批量修改文件夹的名字, 都带上_frms
为了更好的对下次去掉'董终审', 此次也提供支持


In [13]:
import os
import string

r = r'F:\workspace\p7.delta'
os.getcwd()
os.chdir(r)
os.getcwd()

def add_prefix_to_name(ori_name="", prefix="_frms", judge_before_do=True):
    if not ori_name:
        print("no name specified")
        return 
    striped = filter(lambda x: x in string.printable, ori_name)
    ori_name = ''.join(list(striped))
    if judge_before_do:
        new_name = ori_name + prefix if prefix not in ori_name else ori_name
    else:
        new_name = ori_name + prefix
        
    return new_name

#add_prefix_to_name(r)

def change_all_folder_name(root_dir="", rename_func=None):
    if not root_dir or not os.path.isdir(root_dir):
        print("Invaild Root Path")
        return
    if not rename_func:
        print("rename_function must specified")
        return 
    items = os.listdir(root_dir)
    sub_dirs = [i for i in items if os.path.isdir(i)]
    #print(sub_dirs)
    new_dirs = [rename_func(i) for i in sub_dirs]
    #os.rename(sub_dirs[0], new_dirs[0])
    for x,y in zip(sub_dirs, new_dirs):
        print(f"rename {x} to {y}")
        os.rename(x, y)
    return 
    
change_all_folder_name(r, add_prefix_to_name)

rename 07_Plaque_04_YY_01_frms董终审 to 07_Plaque_04_YY_01_frms
rename 07_Plaque_10_YY_02董终审 to 07_Plaque_10_YY_02_frms
rename asdfasj_frms to asdfasj_frms


In [10]:

s = "asdfasj发哈是"
fil = filter(lambda x: x in string.printable, s)
subs = ''.join(list(fil))

asdfasj
